# Laboratorio de Criptoanálisis del Cifrado de Vigenère

**Asignatura:**  Seguridad y Protección de Sistemas Informaticos

**Autor:**  Oscar Fernández Rodriguez, Daniel Fernández Calvo, Diego Adriel Segura Ramírez, Juan Carlos Salas Ariza

**Fecha:**  15/12/2025

## Objetivo
Implementar un laboratorio en Python que permita cifrar, descifrar y
romper criptogramas cifrados mediante el criptosistema de Vigenère.

## Fundamento matemático del cifrado de Vigenère

El cifrado de Vigenère puede modelarse algebraicamente utilizando el
grupo aditivo $\mathbb{Z}_{26}$.

Se define el alfabeto como: 
$$
A={A,B,…,Z},∣A∣=26
$$.

Sea $p_i$ la i-ésima letra del texto plano y $k_j$ la j-ésima letra de 
clave, ambas representadas como elementos de $\mathbb{Z}_{26}$. El
cifrado se define como:

$$
c_i = (p_i + k_{i \bmod m}) \bmod 26
$$

y el descifrado como:

$$
p_i = (c_i - k_{i \bmod m}) \bmod 26
$$

donde $m$ es la longitud de la clave.

Este esquema genera un cifrado polialfabético periódico, base del
criptoanálisis clásico del sistema.

In [4]:
#alfabeto 
alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

#Caracteres especiales
alphSpecials = {
    'Á': 'A',
    'É' : 'E',
    'Í' : 'I',
    'Ó' : 'O',
    'Ú' : 'U',
    'Ä' : 'A',
    'Ë' : 'E',
    'Ï' : 'I',
    'Ö' : 'O',
    'Ü' : 'U',
    'Ñ' : 'GN'
}

#Tamaño del alfabeto
n = len(alphabet)

#Diccionario para  letras : índices 
chNum = {letra: i for i, letra in enumerate(alphabet)}

#Diccionario para índices : letras
numCh = {i: letra for i, letra in enumerate(alphabet)}

#Frecuencias del alfabeto español
alphFreq = {
    'A' : 12.53,
    'B' : 1.42,
    'C' : 4.68,
    'D' : 5.86,
    'E' : 13.68,
    'F' : 0.69,
    'G' : 1.01,
    'H' : 0.70,
    'I' : 6.25,
    'J' : 0.44,
    'K' : 0.02,
    'L' : 4.97,
    'M' : 3.15,
    'N' : 6.71,
    'O' : 8.68,
    'P' : 2.51,
    'Q' : 0.88,
    'R' : 6.87,
    'S' : 7.98,
    'T' : 4.63,
    'U' : 3.93,
    'V' : 0.90,
    'W' : 0.01,
    'X' : 0.22,
    'Y' : 0.90,
    'Z' : 0.52
}

### Kasiski
**Fundamento:** Si una misma secuencia de letras del texto plano se cifra dos o más veces con la misma secuencia de letras de la clave, producirá la misma secuencia de letras en el texto cifrado. Esto ocurre si la longitud de la clave, m, es un divisor de la distancia entre el inicio de las repeticiones en el texto cifrado. <br>
**Procedimiento:** <br>
1) Buscar repeticiones: Se buscan secuencias de 3 o más letras que se repitan en el texto cifrado. <br>
2) Calcular distancias: Para cada repetición encontrada, se calcula la distancia (número de caracteres intermedios) entre el inicio de la primera ocurrencia y el inicio de la siguiente.<br>
3) Encontrar el Máximo Común Divisor (MCD): Se calcula el MCD de todas las distancias obtenidas.<br>
4) Estimación de la clave: El MCD, o un factor de él, es la estimación más probable para la longitud de la clave, m.
   $$
   m=gcd(d1​,d2​,…)
   $$
### Índice de Coincidencia 
**Fundamento:** <br>
El cifrado de Vigenère es un cifrado polialfabético periódico. Esto significa que si la clave tiene una longitud m, el texto cifrado puede dividirse en m subtextos, y cada subtexto es un simple cifrado César (un cifrado monoalfabético). <br><br>
**Definición formal:** <br>
$$
IC = \frac{\sum_{i=A}^{Z} f_i (f_i - 1)}{N (N - 1)}
$$
Donde: <br>

$f_i$: Frecuencia (conteo) de la letra i en el segmento. <br>

$N$: Longitud total del segmento de texto.
### Chi-cuadrado
**Fundamento:** <br>
La prueba $x²$ es una prueba de bondad de ajuste. Compara la distribución de frecuencias de un texto observado (el segmento descifrado con un desplazamiento de prueba) con una distribución de frecuencias esperada. <br><br>
**Definición:** <br>
$$
\chi^2 = \sum_{i=1}^{26} \frac{(O_i - E_i)^2}{E_i}
$$
Donde: <br>

$O_i$: Frecuencia observada (relativa o absoluta) de la letra i en el segmento descifrado. <br>

$E_i$: Frecuencia esperada (relativa o absoluta) de la letra i en español (tomada de tu diccionario alphFreq).

In [5]:
from itertools import chain, groupby  # chain lo usamos para concatenar listas y groupby para agrupar

class Message:
    def _flatten(self,listOfLists):   #convierte una lista de listas en una sola lista
        return list(chain.from_iterable(listOfLists))
    
    def _rBlanks(self, strng):   #Elimina los espacios en blanco de un string y lo convierte a mayusculas
        return ''.join(strng.split()).upper()
    
    def _normalize(self, strng):   #normaliza el string: elimina espacios, convierte a mayusculas y reemplaza caracteres especiales
        s = self._rBlanks(strng)  #convertimos a mayusculas y eliminamos espacios
        accum = []
        for ch in s:
            if ch in alphSpecials:
                accum.append(alphSpecials[ch]) #reemplazamos caracteres especiales
            else:
                accum.append(ch)  #dejamos el caracter tal cual

        return list(filter(lambda x: x in alphabet, self._flatten(accum)))  #filtramos solo los caracteres que estan en el alfabeto
    
    def __init__(self, strng):  #constructor de la clase Message
        x=self._normalize(strng)  #normalizamos el string
        self.content = ''.join(x)  #unimos la lista de caracteres en un string
        self.length = len(self.content)  #longitud del mensaje

    def __str__(self):  # mensaje como string
        return self.content
    
class Encipher(Message):
    
    def __init__(self, text, key): #constructor de la clase Encipher recibe el texto y la clave
        super().__init__(text)  #llamamos al contructor de Message para normalizar el texto
        self.key = ''.join(self._normalize(key))  #normalizamos la clave usando el metodo heredado de Message y lo unios en un solo string por si tuviera varias palabras la clave
        self.key_index=[chNum[k] for k in self.key] #convertimos la clave a indices usando el diccionario chNum

    def cipher(self, mode=True): #Método para cifrar o descifrar el mensaje, mode=True para cifrar, False para descifrar

        textcod = [] #almacenamos el texto cifrado o descifrado
        key_length = len(self.key_index)  #longitud de la clave

        for i, char in enumerate(self.content): #vamos pasando por cada caracter del mensaje
            index = chNum[char] #obtenemos el indice del caracter actual
            key_value =self.key_index[i % key_length] # obtenemos el valor de la clave correspondiente al caracter actual
                                                     # lo hacemos ciclico para que si el mensaje es mas largo que la clave, volvamos a empezar desde el principio de la clave

            if mode: #Comprobamos si estamos cifrando o descifrando
                new_index=(index + key_value) % n #ciframos el caracter sumando el indice del caracter y el valor de la clave
            else:
                new_index=(index - key_value) % n #desciframos el caracter restando el valor de la clave al indice del caracter

            textcod.append(numCh[new_index]) #añadimos el nuevo indice a la lista
        
        return ''.join(textcod)  #unimos la lista en un string y lo devolvemos
    
class BreakVigenere(Message): #Clase encargada de descubrir la clave 

    def __init__(self, text): #constructor de la clase
        super().__init__(text) #llamamos al constructor de la clase Message

    def _index_of_coincidence(self, seg): #Método para obtener el IC (índice de coincidencia) que probabilidad que dos letras al azar sean iguales
        N = len(seg) #Longitud del texto
        count = {char : seg.count(char) for char in alphabet} #Contamos cuantas veces aparece cada letra en el segmento
        return sum(v*(v-1) for v in count.values()) / (N*(N-1)) if N > 1 else 0 #Cálculo del IC

    def estimated_key_length(self, max_length = 20): #Método para estimar la longitud de la clave
        candidates = [] #Lista de candidatos
        for L in range(1, max_length): #Iterar sobre las posibles longitudes de clave
            segments = [self.content[i::L] for i in range(L)] #Toma el segmento desde i y va de L en L
            con_index = sum(self._index_of_coincidence(seg) for seg in segments)/L #Para cada segmento calcula su IC
            candidates.append((L, con_index)) #Lo añadimos a la lista de candidatos
        candidates.sort(key = lambda x: x[1], reverse = True) #Los ordenamos según el IC promedio más alto
        return candidates[0][0] #Seleccionamos la longitud de clave más probable

    def rfrec(self, strng): #método para calcular la frecuencia relativa de cada caracter en un string
        return {k:len(list(g))/len(strng) for k, g in groupby(''.join(sorted(strng)))} #calculamos la frecuencia relativa de cada caracter

    def chiSquared(self, strng): #método para calcular el chi cuadrado de un string
        inventory = dict.fromkeys(alphabet,0) #inicializamos el inventario de frecuencias
        inventory.update(self.rfrec(strng)) #actualizamos el inventario con las frecuencias del texto
        chDegree = [(len(strng)*(inventory[ch]-alphFreq[ch]))**2/alphFreq[ch] for ch in inventory] #calculamos el chi cuadrado
        return sum(chDegree)  #devolvemos la suma del chi cuadrado

    def _best_shift(self, segment): #método para encontrar el mejor desplazamiento para un segmento
        best_shift = 0 #almacenamos el mejor desplazamiento
        best_chi = float('inf') #inicializamos el mejor chi al infinito
        for shift in range(n): #probamos todos los desplazamientos posibles
            decrypted = ''.join(numCh[(chNum[c]-shift) % n] for c in segment) #desciframos el segmento con el desplazamiento actual
            chi = self.chiSquared(decrypted) #calculamos el chi cuadrado del segmento descifrado
            if chi < best_chi: #si el chi cuadrado es mejor que el mejor encontrado hasta ahora
                best_chi = chi #actualizamos el mejor chi
                best_shift = shift #actualizamos el mejor desplazamiento
        return best_shift #devolvemos el mejor desplazamiento encontrado

    def recover_key(self, key_length): # método para recuperara la clave
        key = [] # Clave
        for i in range(key_length): # Recorremos los segmentos con tamañaos i hasta llegar al tamaño de la clave
            segment = self.content[i::key_length] #obtenemos el segmento
            shift = self._best_shift(segment) #Calculamos el mejor desplazamiento
            key.append(numCh[shift]) #añadimos el valor del caracter
        return ''.join(key) #devolvemos la clave sin espacios

    def break_cipher(self): #método que devuelve la clave
        key_length = self.estimated_key_length() # Obtenemos la longitud de la clave
        key = self.recover_key(key_length) # Obtenemos la clave 
        return key #devolvemos la clave encontrada

In [ ]:
import sys

def readTxt(fichero):
    with open(fichero,'r') as f:
        lines = f.readlines()
    accum = [k for k in lines]
    return ''.join(accum)

texto = readTxt("prueba.txt")  #leemos el fichero de entrada
# --- Descifrado automático ---
breaker = BreakVigenere(texto)
key = breaker.break_cipher()

cipher = Encipher(texto, key)
plaintext = cipher.cipher(False)

print("Clave encontrada:", key)
print("\nTexto descifrado:\n")
print(plaintext)
 


Clave encontrada: CAPITAN

Texto descifrado:

SENORDIJOELCAPITANNEMOMOSTRANDOMELOSINSTRUMENTOSCOLGADOSDELASPAREDESDESUCAMAROTEHEAQUILOSAPARATOSEXIGIDOSPORLANAVEGACIONDELNAUTILUSALIGUALQUEENELSALONLOSTENGOAQUIBAJOMISOJOSINDICANDOMEMISITUACIONYMIDIRECCIONEXACTASENMEDIODELOCEANOALGUNOSDEELLOSLESONCONOCIDOSCOMOELTERMOMETROQUEMARCALATEMPERATURAINTERIORDELNAUTILUSELBAROMETROQUEPESAELAIREYPREDICELOSCAMBIOSDETIEMPOELHIGROMETROQUEREGISTRAELGRADODESEQUEDADDELAATMOSFERAELSTORMGLASSCUYAMEZCLAALDESCOMPONERSEANUNCIALAINMINENCIADELASTEMPESTADESLABRUJULAQUEDIRIGEMIRUTAELSEXTANTEQUEPORLAALTURADELSOLMEINDICAMILATITUDLOSCRONOMETROSQUEMEPERMITENCALCULARMILONGITUDYPORULTIMOMISANTEOJOSDEDIAYDENOCHEQUEMESIRVENPARAESCRUTARTODOSLOSPUNTOSDELHORIZONTECUANDOELNAUTILUSEMERGEALASUPERFICIEDELASAGUASSONLOSINSTRUMENTOSHABITUALESDELNAVEGANTEYSUUSOMEESCONOCIDOREPUSEPEROHAYOTROSAQUIQUERESPONDENSINDUDAALASPARTICULARESEXIGENCIASDELNAUTILUSESECUADRANTEQUEVEORECORRIDOPORUNAAGUJAINMOVILNOESUNMANOMETROESUNMANOMETROENEFECTOPUES

## Texto normalizado y con formato
—Señor —dijo el capitán Nemo mostrándome los instrumentos colgados de las paredes de su camarote—, he aquí los aparatos exigidos por la navegación del Nautilus. Al igual que en el salón, los tengo aquí bajo mis ojos, indicándome mi situación y mi dirección exactas en medio del océano.

Algunos de ellos le son conocidos, como el termómetro, que marca la temperatura interior del Nautilus; el barómetro, que pesa el aire y predice los cambios de tiempo; el higrómetro, que registra el grado de sequedad de la atmósfera; el *storm-glass*, cuya mezcla, al descomponerse, anuncia la inminencia de las tempestades; la brújula, que dirige mi ruta; el sextante, que por la altura del sol me indica mi latitud; los cronómetros, que me permiten calcular mi longitud; y, por último, mis anteojos de día y de noche, que me sirven para escrutar todos los puntos del horizonte cuando el Nautilus emerge a la superficie de las aguas.

Son los instrumentos habituales del navegante, y su uso me es conocido —repuse—. Pero hay otros aquí que responden sin duda a las particulares exigencias del Nautilus. Ese cuadrante que veo recorrido por una aguja inmóvil, ¿no es un manómetro?

—Es un manómetro, en efecto —respondió Nemo—, puesto en comunicación con el agua, cuya presión exterior indica también la profundidad a la que se mantiene mi aparato.

## ¿Es seguro a día de hoy Vigenère?
El cifrado de Vigenère fue considerado irrompible durante siglos, pero su seguridad colapsó gracias al análisis de periodicidad, especialmente con el método del Índice de Coincidencia (IC) y el Test de Kasiski. Hoy en día, se considera completamente inseguro y su ruptura se realiza en segundos. <br><br>
La razón fundamental de su debilidad radica en dos conceptos clave que violan los principios de la criptografía moderna: periodicidad y reutilización de clave.